In [1]:
import torch
import dataset_cleaning
import polars as pl

In [2]:
df, CHAMP_TO_IDX = dataset_cleaning.generate_dataset()

In [3]:
# Define the function
def standardize_matchup(blue, red):
    blue_str, red_str = '-'.join(map(str, sorted(blue))), '-'.join(map(str, sorted(red)))
    
    if blue_str < red_str:
        return f"{blue_str}/{red_str}"
    else:
        return f"{red_str}/{blue_str}"
    
def standardize_matchup(blue, red):
    return '-'.join(map(str, sorted(blue + red)))


In [4]:
df = df.with_columns(
   pl.struct(["Blue_champions", "Red_champions"])\
   .apply(lambda x: standardize_matchup(x['Blue_champions'], x['Red_champions']))\
   .alias('matchup_id'))

In [5]:
df.n_unique('matchup_id'), df.shape[0]

(41964, 42140)

In [6]:
from torch.utils.data import Dataset

class LeagueOfLegendsDataset(Dataset):
    def __init__(self, type="train"):
        
        assert type in ["train", "test"]
        
        self.blue_champions = torch.load(f=f"dataset\\blue_champions_{type}.pt")
        self.red_champions = torch.load(f=f"dataset\\red_champions_{type}.pt")
        self.result = torch.load(f=f"dataset\\result_{type}.pt")
        print("Dataset loaded")

    def __len__(self):
        assert len(self.blue_champions) == len(self.red_champions) == len(self.result)
        return len(self.blue_champions)

    def __getitem__(self, idx):
        return self.blue_champions[idx], self.red_champions[idx], self.result[idx]


In [7]:
from torch.utils.data import DataLoader

dataset_train = LeagueOfLegendsDataset(type="train")
data_loader_train = DataLoader(dataset_train, batch_size=16, shuffle=True)

dataset_test = LeagueOfLegendsDataset(type="test")
data_loader_test = DataLoader(dataset_test, batch_size=16, shuffle=True)

Dataset loaded
Dataset loaded


In [5]:
len(dataset_train), len(dataset_test)

(5987, 1496)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class TeamPredictor(nn.Module):
    def __init__(self, n_champions, n_dim):
        super(TeamPredictor, self).__init__()
        
        # Define an embedding layer for champion synergies and good-against properties
        self.synergy_embeddings = nn.Embedding(n_champions, n_dim)
    
    def forward(self, blue_team, red_team):
        # Embed the blue team champions
        blue_team_synergies = self.synergy_embeddings(blue_team)
        red_team_synergies = self.synergy_embeddings(red_team)
        
        # M * M^T, M = (batch, n_champions, dim)
        blue_team_synergies = torch.matmul(blue_team_synergies, blue_team_synergies.transpose(1, 2))
        red_team_synergies = torch.matmul(red_team_synergies, red_team_synergies.transpose(1, 2))
        
        # Sum whole everything but the diagonal
        final_blue_score = torch.sum(blue_team_synergies, dim=(1, 2)) - torch.sum(torch.diagonal(blue_team_synergies, dim1=1, dim2=2), dim=1)
        final_red_score = torch.sum(red_team_synergies, dim=(1, 2)) - torch.sum(torch.diagonal(red_team_synergies, dim1=1, dim2=2), dim=1)
        
        # Concatenate the blue and red team scores
        scores = torch.cat((final_red_score.unsqueeze(-1), final_blue_score.unsqueeze(-1)), dim=1)
        
        return scores


In [11]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Import for the scheduler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TeamPredictor(n_champions=len(CHAMP_TO_IDX), n_dim=125).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)  # L2 regularization

# Initialize the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

criterion = CrossEntropyLoss()

n_epochs = 100

best_test_loss = float('inf')

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    train_correct = 0  # Track correct predictions for training
    train_samples = 0  # Track total number of training samples
    for i, data in enumerate(data_loader_train):
        blue_champions, red_champions, labels = data
        blue_champions, red_champions, labels = blue_champions.to(device), red_champions.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(blue_champions, red_champions)
        _, predicted = torch.max(outputs.data, 1)
        train_samples += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(data_loader_train)
    train_accuracy = 100 * train_correct / train_samples  # Compute training accuracy
    print(f"Epoch: {epoch + 1}, Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy} %")

    model.eval()
    total_correct = 0
    total_samples = 0
    running_test_loss = 0.0
    with torch.no_grad():
        for data in data_loader_test:
            blue_champions, red_champions, labels = data
            blue_champions, red_champions, labels = blue_champions.to(device), red_champions.to(device), labels.to(device)
            outputs = model(blue_champions, red_champions)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()
            test_loss = criterion(outputs, labels)
            running_test_loss += test_loss.item()

    avg_test_loss = running_test_loss / len(data_loader_test)
    print('Test Accuracy: {} %, Test Loss: {}'.format(100 * total_correct / total_samples, avg_test_loss))

    # Step the scheduler
    scheduler.step(avg_test_loss)


Epoch: 1, Train Loss: 29.689509372321925, Train Accuracy: 50.32332700522069 %
Test Accuracy: 50.93735168485999 %, Test Loss: 22.02618673178005
Epoch: 2, Train Loss: 15.955127936151845, Train Accuracy: 52.51839107736118 %
Test Accuracy: 50.735643094447084 %, Test Loss: 12.959794810646173
Epoch: 3, Train Loss: 8.374987195878443, Train Accuracy: 54.3812292358804 %
Test Accuracy: 50.842429995253916 %, Test Loss: 7.3090223187061145
Epoch: 4, Train Loss: 4.172987941292737, Train Accuracy: 55.8999762695776 %
Test Accuracy: 50.628856193640246 %, Test Loss: 3.7905131746967107
Epoch: 5, Train Loss: 1.9858257189237454, Train Accuracy: 57.24964404366398 %
Test Accuracy: 51.649264356905555 %, Test Loss: 1.8197719882855605
Epoch: 6, Train Loss: 1.021934645007332, Train Accuracy: 58.13360227812055 %
Test Accuracy: 51.74418604651163 %, Test Loss: 0.9902837634425914
Epoch: 7, Train Loss: 0.7365514135892548, Train Accuracy: 57.819174181300426 %
Test Accuracy: 51.42382534409113 %, Test Loss: 0.7610307197

KeyboardInterrupt: 